In [2]:
import pandas as pd
import os

In [3]:
os.chdir("..")

In [4]:
os.getcwd()

'c:\\Users\\user\\Downloads\\ten_academy\\week8\\Causal_Inference'

In [5]:
# Load the datasets
completed_orders_no_outliers = pd.read_csv('data/completed_orders_no_outliers.csv')

In [6]:
completed_orders_no_outliers

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,Trip Duration(minutes),Day of Week,Hour of Day,Origin Lat,Origin Lon,Destination Lat,Destination Lon,Trip Distance (km),Trip Date
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,1.550000,3,7,6.508813,3.377403,6.650970,3.345031,16.206549,2021-07-01
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,29.400000,3,6,6.431671,3.455538,6.428081,3.472189,1.882675,2021-07-01
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,41.350000,3,6,6.631679,3.338898,6.508324,3.359040,13.895773,2021-07-01
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,13.583333,3,7,6.572757,3.367708,6.584881,3.361407,1.517186,2021-07-01
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,3,9,6.601042,3.276634,6.450107,3.391615,21.048262,2021-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505318,1637696,"6.448218499999999,3.4772075","6.437787399999999,3.481670199999999",2021-12-30 20:35:06,2021-12-30 21:02:59,27.883333,3,20,6.448218,3.477208,6.437787,3.481670,1.260348,2021-12-30
505319,1637702,"6.442320899999999,3.4736868","6.436589333407897,3.5559738188407835",2021-12-30 20:48:13,2021-12-30 21:43:49,55.600000,3,20,6.442321,3.473687,6.436589,3.555974,9.114481,2021-12-30
505320,1637704,"6.4281982,3.492248","6.448088500000001,3.4775747",2021-12-30 20:51:45,2021-12-30 21:41:32,49.783333,3,20,6.428198,3.492248,6.448089,3.477575,2.742308,2021-12-30
505321,1637705,"6.5869296,3.3632966","6.637906899999999,3.3339515",2021-12-30 20:48:50,2021-12-30 21:08:28,19.633333,3,20,6.586930,3.363297,6.637907,3.333951,6.529710,2021-12-30


### Feature Engineering

Create new features based on time and location, such as trip duration, day of the week, hour of the day, and whether the trip occurred during a holiday or special event.
Compute distances, driving speed, and other relevant metrics.

- **Day of the Week** Numeric representation of the day (0 = Monday, 6 = Sunday).
- **Hour of the Day** Hour of the day when the trip started.

### Distance Calculation

Compute distances between trip origin and destination using latitude and longitude coordinates. (Note: We may need additional data for latitude and longitude of trip origins and destinations if not already included.)

In [7]:
# Inspect the contents of the 'Trip Origin' and 'Trip Destination' columns
trip_origins_sample = completed_orders_no_outliers['Trip Origin'].head(10)
trip_destinations_sample = completed_orders_no_outliers['Trip Destination'].head(10)

trip_origins_sample, trip_destinations_sample


(0    6.508813001668548,3.37740316890347
 1                   6.4316714,3.4555375
 2           6.631679399999999,3.3388976
 3           6.572757200000001,3.3677082
 4                   6.6010417,3.2766339
 5           6.565087699999999,3.3844415
 6                   6.6636484,3.3082058
 7                   6.4308171,3.4341552
 8           6.499156300000001,3.3585173
 9           6.535043099999999,3.3316238
 Name: Trip Origin, dtype: object,
 0        6.650969799999999,3.3450307
 1    6.4280814653326,3.4721885847586
 2        6.508324099999999,3.3590397
 3        6.584881099999999,3.3614073
 4                6.4501069,3.3916154
 5        6.499696300000001,3.3509075
 6                 6.6185421,3.301634
 7        6.435460000000001,3.4846547
 8                6.4280911,3.5157172
 9        6.500039999999999,3.3248596
 Name: Trip Destination, dtype: object)

In [8]:
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Split latitude and longitude for origins and destinations
completed_orders_no_outliers[['Origin Lat', 'Origin Lon']] = completed_orders_no_outliers['Trip Origin'].str.split(',', expand=True).astype(float)
completed_orders_no_outliers[['Destination Lat', 'Destination Lon']] = completed_orders_no_outliers['Trip Destination'].str.split(',', expand=True).astype(float)

# Calculate the distance between origins and destinations
completed_orders_no_outliers['Trip Distance (km)'] = haversine_distance(
    completed_orders_no_outliers['Origin Lat'], completed_orders_no_outliers['Origin Lon'],
    completed_orders_no_outliers['Destination Lat'], completed_orders_no_outliers['Destination Lon']
)

# Display the first few rows to confirm the new feature
distance_sample = completed_orders_no_outliers[['Trip ID', 'Origin Lat', 'Origin Lon', 'Destination Lat', 'Destination Lon', 'Trip Distance (km)']].head()

distance_sample


,Trip ID,Origin Lat,Origin Lon,Destination Lat,Destination Lon,Trip Distance (km)
0,391996,6.508813,3.377403,6.650970,3.345031,16.206549
1,391997,6.431671,3.455538,6.428081,3.472189,1.882675
2,391998,6.631679,3.338898,6.508324,3.359040,13.895773
3,391999,6.572757,3.367708,6.584881,3.361407,1.517186
4,392001,6.601042,3.276634,6.450107,3.391615,21.048262


In [9]:
completed_orders_no_outliers.dtypes

Trip ID                     int64
Trip Origin                object
Trip Destination           object
Trip Start Time            object
Trip End Time              object
Trip Duration(minutes)    float64
Day of Week                 int64
Hour of Day                 int64
Origin Lat                float64
Origin Lon                float64
Destination Lat           float64
Destination Lon           float64
Trip Distance (km)        float64
Trip Date                  object
dtype: object

In [10]:

completed_orders_no_outliers['Trip Start Time'] = pd.to_datetime(completed_orders_no_outliers['Trip Start Time'])
completed_orders_no_outliers['Trip End Time'] = pd.to_datetime(completed_orders_no_outliers['Trip End Time'])

In [11]:
completed_orders_no_outliers['Origin Lat'][0].dtype, completed_orders_no_outliers['Origin Lon'][0].dtype

(dtype('float64'), dtype('float64'))

In [ ]:
# Import module
from geopy.geocoders import Photon
geolocator = Photon(user_agent="measurements")
 
# Assign Latitude & Longitude
Latitude = completed_orders_no_outliers['Origin Lat'][0]
Longitude = completed_orders_no_outliers['Origin Lon'][0]
 
# Displaying Latitude and Longitude
print("Latitude: ", Latitude)
print("Longitude: ", Longitude)
 
# Get location with geocode
location = geolocator.reverse(f"{Latitude},{Longitude}")
 
# Display location
print("\nLocation of the given Latitude and Longitude:")
print(location)

In [ ]:
# Import module
from geopy.geocoders import Photon
geolocator = Photon(user_agent="measurements")
 
# Assign location
location_coordinates = completed_orders_no_outliers['Trip Origin'][0]
 
# Displaying Location
print("Location: ", location_coordinates)
 
# Get location with geocode
location_name = geolocator.reverse(location_coordinates)
 
# Display location
print("\nLocation of the given Latitude and Longitude:")
print(location_name)

In [12]:
completed_orders_no_outliers['Trip Origin'].isna().sum()

0

In [13]:
type(completed_orders_no_outliers['Trip Origin'])

pandas.core.series.Series

In [ ]:
completed_orders_no_outliers['Location Name'] = completed_orders_no_outliers['Trip Origin'].apply(lambda x: geolocator.reverse(x, timeout=20))

In [ ]:
import pandas as pd
from geopy.geocoders import Photon

# Get the unique values of the "Trip Origin" column
unique_values = completed_orders_no_outliers['Trip Origin'].unique()

print("The unique values in the 'Trip Origin' column are: ",len(unique_values))
# Create an empty list to store the reverse geocoded values
reverse_geocoded_values = []

# Create a geocoder object
geolocator = Photon(user_agent="measurements")

# Iterate over each unique value and perform reverse geocoding
i = 1
for value in unique_values:
    location = geolocator.reverse(value)
    if location:
        reverse_geocoded_values.append(location.address)
        print(i, ". Address is :: ", location.address)
    else:
        reverse_geocoded_values.append(None)
        print(i, ". Address Not Found :: ", value)
    i = i + 1
# Create a new column in the DataFrame with the reverse geocoded values
completed_orders_no_outliers['Location Name'] = pd.Series(reverse_geocoded_values)

In [ ]:
completed_orders_no_outliers.nunique()

In [ ]:
from geopy.geocoders import Photon
geolocator = Photon(user_agent="measurements")

location = geolocator.geocode("175 5th Avenue NYC")
print(location)

In [ ]:
completed_orders_no_outliers['Trip Origin'][0]

In [ ]:
from pandas import Timestamp


m: Timestamp = completed_orders_no_outliers['Trip Start Time'][0]
m.to_pydatetime()

In [45]:
from meteostat import Point, Daily, Hourly

# Define location for Lagos, Nigeria
lagos = Point(6.5244, 3.3792)

# Define time range for the data
start_date = completed_orders_no_outliers['Trip Start Time'].min().to_pydatetime()
# decrease one day from the start date
start_date = start_date - pd.Timedelta(days=1)
end_date = completed_orders_no_outliers['Trip Start Time'].max().to_pydatetime()
start_date, end_date
# Get daily weather data for Lagos
weather_data = Daily(lagos, start_date, end_date)
weather_data = weather_data.fetch()

# Display the first few rows of the weather data
weather_data.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2021-07-01,27.8,24.0,33.0,0.0,NaN,253.0,10.1,NaN,1014.7,NaN
2021-07-02,27.5,24.9,31.2,NaN,NaN,248.0,9.5,NaN,1014.0,NaN
2021-07-03,25.9,23.0,29.5,NaN,NaN,258.0,11.0,NaN,1014.6,NaN
2021-07-04,25.7,22.0,31.0,NaN,NaN,257.0,8.8,NaN,1015.5,NaN
2021-07-05,27.6,24.0,32.0,NaN,NaN,242.0,8.8,NaN,1015.6,NaN


In [46]:
weather_data = Hourly(lagos, start_date, end_date)

In [47]:
weather_data = weather_data.fetch()

In [49]:
weather_data.head()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2021-06-30 07:00:00,25.2,23.6,91.0,NaN,NaN,338.0,3.7,NaN,1015.8,NaN,3.0
2021-06-30 08:00:00,26.3,24.2,88.0,NaN,NaN,317.0,3.7,NaN,1016.3,NaN,3.0
2021-06-30 09:00:00,27.4,24.7,85.0,NaN,NaN,309.0,5.5,NaN,1016.7,NaN,3.0
2021-06-30 10:00:00,28.4,24.6,80.0,NaN,NaN,301.0,5.5,NaN,1016.2,NaN,3.0
2021-06-30 11:00:00,30.0,23.9,70.0,NaN,NaN,180.0,11.2,NaN,1016.0,NaN,3.0


We can see that only those weather condition are available this lets add them as column

In [ ]:
"2021-07-01 07:00:00"

In [50]:
weather_data['weather_condition'] = weather_data['coco'].map({
    1: 'Clear',
    2: 'Fair',
    3: 'Cloudy',
    4: 'Overcast',
    5: 'Fog',
    6: 'Freezing Fog',
    7: 'Light Rain',
    8: 'Rain',
    9: 'Heavy Rain',
    10: 'Freezing Rain',
    11: 'Heavy Freezing Rain',
    12: 'Sleet',
    13: 'Heavy Sleet',
    14: 'Light Snowfall',
    15: 'Snowfall',
    16: 'Heavy Snowfall',
    17: 'Rain Shower',
    18: 'Heavy Rain Shower',
    19: 'Sleet Shower',
    20: 'Heavy Sleet Shower',
    21: 'Snow Shower',
    22: 'Heavy Snow Shower',
    23: 'Lightning',
    24: 'Hail',
    25: 'Thunderstorm',
    26: 'Heavy Thunderstorm',
    27: 'Storm'
})

weather_data['Is Cloudy'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Cloudy' else 0)
weather_data['Is Fair'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Fair' else 0)
weather_data['Is Fog'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Fog' else 0)
weather_data['Is Thunderstorm'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Thunderstorm' else 0)
weather_data['Is Heavy Rain Shower'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Heavy Rain Shower' else 0)
weather_data['Is Light Rain'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Light Rain' else 0)
weather_data['Is Rain'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Rain' else 0)
weather_data['Is Overcast'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Overcast' else 0)
weather_data['Is Heavy Rain'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Heavy Rain' else 0)
weather_data['Is Clear'] = weather_data['weather_condition'].apply(lambda x: 1 if x == 'Clear' else 0)

In [ ]:
start_date, end_date

In [21]:
len(weather_data)

8356

In [ ]:
# Plot line chart including average, minimum and maximum temperature
import matplotlib.pyplot as plt
weather_data.plot(y=['tavg', 'tmin', 'tmax'])
plt.show()

In [ ]:
# Save the weather data to a CSV file
weather_data.to_csv('data/weather_data_hourly_lagos.csv')

In [63]:
# Merge the weather data with trip data on the date
completed_orders_no_outliers['Trip Date'] = completed_orders_no_outliers['Trip Start Time'].dt.strftime('%Y-%m-%d %H:00:00')
weather_data['date'] = weather_data.index

In [57]:
weather_data[['date', 'temp']]

,date,temp
time,,
2021-06-30 07:00:00,2021-06-30 07:00:00,25.2
2021-06-30 08:00:00,2021-06-30 08:00:00,26.3
2021-06-30 09:00:00,2021-06-30 09:00:00,27.4
2021-06-30 10:00:00,2021-06-30 10:00:00,28.4
2021-06-30 11:00:00,2021-06-30 11:00:00,30.0
...,...,...
2022-06-13 06:00:00,2022-06-13 06:00:00,24.9
2022-06-13 07:00:00,2022-06-13 07:00:00,26.0
2022-06-13 08:00:00,2022-06-13 08:00:00,28.0


In [52]:
completed_orders_no_outliers.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,Trip Duration(minutes),Day of Week,Hour of Day,Origin Lat,Origin Lon,Destination Lat,Destination Lon,Trip Distance (km),Trip Date
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,1.550000,3,7,6.508813,3.377403,6.650970,3.345031,16.206549,2021-07-01 07
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,29.400000,3,6,6.431671,3.455538,6.428081,3.472189,1.882675,2021-07-01 06
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,41.350000,3,6,6.631679,3.338898,6.508324,3.359040,13.895773,2021-07-01 06
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,13.583333,3,7,6.572757,3.367708,6.584881,3.361407,1.517186,2021-07-01 07
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,3,9,6.601042,3.276634,6.450107,3.391615,21.048262,2021-07-01 09


In [61]:
completed_orders_no_outliers.head()[['Destination Lon','Trip Distance (km)','Trip Date']]
# weather date 2021-06-30 07:00:00

,Destination Lon,Trip Distance (km),Trip Date
0,3.345031,16.206549,2021-07-01 07:00:00
1,3.472189,1.882675,2021-07-01 06:00:00
2,3.359040,13.895773,2021-07-01 06:00:00
3,3.361407,1.517186,2021-07-01 07:00:00
4,3.391615,21.048262,2021-07-01 09:00:00


In [69]:
completed_orders_no_outliers.dtypes

Trip ID                            int64
Trip Origin                       object
Trip Destination                  object
Trip Start Time           datetime64[ns]
Trip End Time             datetime64[ns]
Trip Duration(minutes)           float64
Day of Week                        int64
Hour of Day                        int64
Origin Lat                       float64
Origin Lon                       float64
Destination Lat                  float64
Destination Lon                  float64
Trip Distance (km)               float64
Trip Date                         object
dtype: object

In [27]:
# get the date and the hour part only from a row data that looks like "2021-07-01 07:28:04"
completed_orders_no_outliers['Trip Start Time'].dt.strftime('%Y-%m-%d %H:00:00')

0         2021-07-01 07:00:00
1         2021-07-01 06:00:00
2         2021-07-01 06:00:00
3         2021-07-01 07:00:00
4         2021-07-01 09:00:00
                 ...         
505318    2021-12-30 20:00:00
505319    2021-12-30 20:00:00
505320    2021-12-30 20:00:00
505321    2021-12-30 20:00:00
505322    2021-12-30 20:00:00
Name: Trip Start Time, Length: 505323, dtype: object

In [ ]:

# weather_data['date'] = pd.to_datetime(weather_data['date'])
# completed_orders_no_outliers['Trip End Time'] = pd.to_datetime(completed_orders_no_outliers['Trip End Time'])

In [66]:
weather_data.dtypes

temp                    float64
dwpt                    float64
rhum                    float64
prcp                    float64
snow                    float64
wdir                    float64
wspd                    float64
wpgt                    float64
pres                    float64
tsun                    float64
coco                    float64
weather_condition        object
Is Cloudy                 int64
Is Fair                   int64
Is Fog                    int64
Is Thunderstorm           int64
Is Heavy Rain Shower      int64
Is Light Rain             int64
Is Rain                   int64
Is Overcast               int64
Is Heavy Rain             int64
Is Clear                  int64
date                     object
dtype: object

In [65]:

# convert date to object
weather_data['date'] = weather_data['date'].dt.strftime('%Y-%m-%d %H:00:00')

In [67]:
weather_data.head(1)
# [['date', 'temp']]

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,...,Is Fair,Is Fog,Is Thunderstorm,Is Heavy Rain Shower,Is Light Rain,Is Rain,Is Overcast,Is Heavy Rain,Is Clear,date
time,,,,,,,,,,,,,,,,,,,,,
2021-06-30 07:00:00,25.2,23.6,91.0,NaN,NaN,338.0,3.7,NaN,1015.8,NaN,...,0,0,0,0,0,0,0,0,0,2021-06-30 07:00:00


In [77]:
# Merge on the 'Trip Date' and 'date' columns
merged_data = pd.merge(completed_orders_no_outliers, weather_data, left_on='Trip Date', right_on='date', how='left')

In [78]:
# Display the first few rows of the merged data
merged_data.head(1)

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,Trip Duration(minutes),Day of Week,Hour of Day,Origin Lat,Origin Lon,...,Is Fair,Is Fog,Is Thunderstorm,Is Heavy Rain Shower,Is Light Rain,Is Rain,Is Overcast,Is Heavy Rain,Is Clear,date
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,1.55,3,7,6.508813,3.377403,...,0,0,0,0,0,0,0,0,0,2021-07-01 07:00:00


In [79]:
merged_data.shape

(505323, 37)

In [80]:
merged_data.isna().sum()/len(merged_data) * 100

Trip ID                     0.000000
Trip Origin                 0.000000
Trip Destination            0.000000
Trip Start Time             0.000000
Trip End Time               0.000000
Trip Duration(minutes)      0.000000
Day of Week                 0.000000
Hour of Day                 0.000000
Origin Lat                  0.000000
Origin Lon                  0.000000
Destination Lat             0.000000
Destination Lon             0.000000
Trip Distance (km)          0.000000
Trip Date                   0.000000
temp                        0.000000
dwpt                        0.000000
rhum                        0.000000
prcp                       99.999406
snow                      100.000000
wdir                        0.000000
wspd                        0.000000
wpgt                      100.000000
pres                        0.000000
tsun                      100.000000
coco                        0.000198
weather_condition           0.000198
Is Cloudy                   0.000000
I

In [81]:
# drop columns 
merged_data.drop(['dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wpgt', 'tsun', 'coco', 'weather_condition', 'date'], axis=1, inplace=True)

In [84]:
merged_data.isna().sum()/len(merged_data) * 100

Trip ID                         0.0
Trip Origin                     0.0
Trip Destination                0.0
Trip Start Time                 0.0
Trip End Time                   0.0
Trip Duration(minutes)          0.0
Day of Week                     0.0
Hour of Day                     0.0
Origin Lat                      0.0
Origin Lon                      0.0
Destination Lat                 0.0
Destination Lon                 0.0
Trip Distance (km)              0.0
Trip Date                       0.0
Average Temperature(Celcius)    0.0
Wind Speed(km/h)                0.0
Air Pressure(hPa)               0.0
Is Cloudy                       0.0
Is Fair                         0.0
Is Fog                          0.0
Is Thunderstorm                 0.0
Is Heavy Rain Shower            0.0
Is Light Rain                   0.0
Is Rain                         0.0
Is Overcast                     0.0
Is Heavy Rain                   0.0
Is Clear                        0.0
dtype: float64

In [ ]:
merged_data.isnull().sum()/len(merged_data) * 100

In [83]:
# rename columns tmin	tmax	prcp	snow	wdir	wspd	wpgt	pres	tsun

merged_data.rename(columns={'wspd': 'Wind Speed(km/h)', 
                            'temp': 'Average Temperature(Celcius)',
                            'pres': 'Air Pressure(hPa)',}, inplace=True)

In [156]:
merged_data.dtypes

Trip ID                                  int64
Trip Origin                             object
Trip Destination                        object
Trip Start Time                 datetime64[ns]
Trip End Time                   datetime64[ns]
Trip Duration(minutes)                 float64
Day of Week                              int64
Hour of Day                              int64
Origin Lat                             float64
Origin Lon                             float64
Destination Lat                        float64
Destination Lon                        float64
Trip Distance (km)                     float64
Trip Date                               object
Average Temperature(Celcius)           float64
Wind Speed(km/h)                       float64
Air Pressure(hPa)                      float64
Is Cloudy                                int64
Is Fair                                  int64
Is Fog                                   int64
Is Thunderstorm                          int64
Is Heavy Rain

In [169]:
import holidays
nigerian_holidays = holidays.country_holidays(country='NG', years=[2021, 2022])

In [150]:
date_to_check = str(merged_data['Trip Date'][1000])

In [160]:
date_to_check = '2021-01-01'

In [126]:
nigerian_holidays.items()

dict_items([(datetime.date(2021, 1, 1), "New Year's Day"), (datetime.date(2021, 4, 2), 'Good Friday'), (datetime.date(2021, 4, 5), 'Easter Monday'), (datetime.date(2021, 5, 1), "Workers' Day"), (datetime.date(2021, 6, 12), 'Democracy Day'), (datetime.date(2021, 10, 1), 'Independence Day'), (datetime.date(2021, 12, 25), 'Christmas Day'), (datetime.date(2021, 12, 26), 'Boxing Day'), (datetime.date(2021, 5, 13), 'Eid-el-Fitr (estimated)'), (datetime.date(2021, 5, 14), 'Eid-el-Fitr Holiday (estimated)'), (datetime.date(2021, 7, 20), 'Eid-el-Kabir (estimated)'), (datetime.date(2021, 7, 21), 'Eid-el-Kabir Holiday (estimated)'), (datetime.date(2021, 10, 18), 'Eid-el-Mawlid (estimated)'), (datetime.date(2021, 5, 3), "Workers' Day (observed)"), (datetime.date(2021, 6, 14), 'Democracy Day (observed)'), (datetime.date(2021, 12, 27), 'Christmas Day (observed)'), (datetime.date(2021, 12, 28), 'Boxing Day (observed)'), (datetime.date(2022, 1, 1), "New Year's Day"), (datetime.date(2022, 4, 15), 'Good

In [177]:
merged_data.dtypes

Trip ID                                  int64
Trip Origin                             object
Trip Destination                        object
Trip Start Time                 datetime64[ns]
Trip End Time                   datetime64[ns]
Trip Duration(minutes)                 float64
Day of Week                              int64
Hour of Day                              int64
Origin Lat                             float64
Origin Lon                             float64
Destination Lat                        float64
Destination Lon                        float64
Trip Distance (km)                     float64
Trip Date                               object
Average Temperature(Celcius)           float64
Wind Speed(km/h)                       float64
Air Pressure(hPa)                      float64
Is Cloudy                                int64
Is Fair                                  int64
Is Fog                                   int64
Is Thunderstorm                          int64
Is Heavy Rain

In [162]:
for date, holidays in nigerian_holidays.items():
    print(type(date), date, holidays)

<class 'datetime.date'> 2021-01-01 New Year's Day
<class 'datetime.date'> 2021-04-02 Good Friday
<class 'datetime.date'> 2021-04-05 Easter Monday
<class 'datetime.date'> 2021-05-01 Workers' Day
<class 'datetime.date'> 2021-06-12 Democracy Day
<class 'datetime.date'> 2021-10-01 Independence Day
<class 'datetime.date'> 2021-12-25 Christmas Day
<class 'datetime.date'> 2021-12-26 Boxing Day
<class 'datetime.date'> 2021-05-13 Eid-el-Fitr (estimated)
<class 'datetime.date'> 2021-05-14 Eid-el-Fitr Holiday (estimated)
<class 'datetime.date'> 2021-07-20 Eid-el-Kabir (estimated)
<class 'datetime.date'> 2021-07-21 Eid-el-Kabir Holiday (estimated)
<class 'datetime.date'> 2021-10-18 Eid-el-Mawlid (estimated)
<class 'datetime.date'> 2021-05-03 Workers' Day (observed)
<class 'datetime.date'> 2021-06-14 Democracy Day (observed)
<class 'datetime.date'> 2021-12-27 Christmas Day (observed)
<class 'datetime.date'> 2021-12-28 Boxing Day (observed)
<class 'datetime.date'> 2022-01-01 New Year's Day
<class 'd

In [173]:
len(nigerian_holidays.items())

38

In [187]:
merged_data['Trip Date'][0]

datetime.date(2021, 7, 1)

In [186]:
# nigerian_holidays.get('2021-4-2')
"2022, 10, 01" in nigerian_holidays

True

In [193]:
merged_data['Is Holiday'] = merged_data['Trip Date'].apply(lambda x: int(x in nigerian_holidays))

In [195]:
merged_data.dtypes

Trip ID                                  int64
Trip Origin                             object
Trip Destination                        object
Trip Start Time                 datetime64[ns]
Trip End Time                   datetime64[ns]
Trip Duration(minutes)                 float64
Day of Week                              int64
Hour of Day                              int64
Origin Lat                             float64
Origin Lon                             float64
Destination Lat                        float64
Destination Lon                        float64
Trip Distance (km)                     float64
Trip Date                               object
Average Temperature(Celcius)           float64
Wind Speed(km/h)                       float64
Air Pressure(hPa)                      float64
Is Cloudy                                int64
Is Fair                                  int64
Is Fog                                   int64
Is Thunderstorm                          int64
Is Heavy Rain

In [196]:

merged_data.drop(['Trip Origin','Trip Destination', 'Trip Date'], axis=1, inplace=True)

In [199]:
merged_data.isna().sum()/len(merged_data) * 100

Trip ID                         0.0
Trip Start Time                 0.0
Trip End Time                   0.0
Trip Duration(minutes)          0.0
Day of Week                     0.0
Hour of Day                     0.0
Origin Lat                      0.0
Origin Lon                      0.0
Destination Lat                 0.0
Destination Lon                 0.0
Trip Distance (km)              0.0
Average Temperature(Celcius)    0.0
Wind Speed(km/h)                0.0
Air Pressure(hPa)               0.0
Is Cloudy                       0.0
Is Fair                         0.0
Is Fog                          0.0
Is Thunderstorm                 0.0
Is Heavy Rain Shower            0.0
Is Light Rain                   0.0
Is Rain                         0.0
Is Overcast                     0.0
Is Heavy Rain                   0.0
Is Clear                        0.0
Is Holiday                      0.0
dtype: float64

In [198]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505323 entries, 0 to 505322
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Trip ID                       505323 non-null  int64         
 1   Trip Start Time               505323 non-null  datetime64[ns]
 2   Trip End Time                 505323 non-null  datetime64[ns]
 3   Trip Duration(minutes)        505323 non-null  float64       
 4   Day of Week                   505323 non-null  int64         
 5   Hour of Day                   505323 non-null  int64         
 6   Origin Lat                    505323 non-null  float64       
 7   Origin Lon                    505323 non-null  float64       
 8   Destination Lat               505323 non-null  float64       
 9   Destination Lon               505323 non-null  float64       
 10  Trip Distance (km)            505323 non-null  float64       
 11  Average Tempe

In [197]:
merged_data.tail(1)

,Trip ID,Trip Start Time,Trip End Time,Trip Duration(minutes),Day of Week,Hour of Day,Origin Lat,Origin Lon,Destination Lat,Destination Lon,...,Is Fair,Is Fog,Is Thunderstorm,Is Heavy Rain Shower,Is Light Rain,Is Rain,Is Overcast,Is Heavy Rain,Is Clear,Is Holiday
505322,1637709,2021-12-30 20:55:38,2021-12-30 22:25:00,89.366667,3,20,6.64721,3.485149,6.454915,3.55584,...,1,0,0,0,0,0,0,0,0,0


In [203]:
merged_data['Is Weekend'] = merged_data['Day of Week'].apply(lambda x: 1 if x in [5, 6] else 0)

In [205]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505323 entries, 0 to 505322
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Trip ID                       505323 non-null  int64         
 1   Trip Start Time               505323 non-null  datetime64[ns]
 2   Trip End Time                 505323 non-null  datetime64[ns]
 3   Trip Duration(minutes)        505323 non-null  float64       
 4   Day of Week                   505323 non-null  int64         
 5   Hour of Day                   505323 non-null  int64         
 6   Origin Lat                    505323 non-null  float64       
 7   Origin Lon                    505323 non-null  float64       
 8   Destination Lat               505323 non-null  float64       
 9   Destination Lon               505323 non-null  float64       
 10  Trip Distance (km)            505323 non-null  float64       
 11  Average Tempe

In [207]:
merged_data.to_csv('data/complete_orders_features_engineered.csv', index=False)